# Project: Customer classification (Online Retail II)

- Date: August 5 2025 -

- Data: This Online Retail II data set contains all the transactions occurring for a UK-based and registered, non-store online retail between 01/12/2009 and 09/12/2011.The company mainly sells unique all-occasion gift-ware. Many customers of the company are wholesalers.

- Description: In this project our goal will be to use this data to detect some similarities between the customers,divide them in some groups with some particular habits or behaviors. This project could be very interresting for marketing strategies. 

In [ ]:
import pandas as pd

# Download the excel file
df = pd.read_excel('/home/christian/ProjetsPerso/IA/MachineLearning/Customer_Classification/data/online_retail_II.xlsx',
                    sheet_name="Year 2010-2011",
                   dtype={"Invoice": str, "StockCode": str, "Customer ID": str},
                   parse_dates=["InvoiceDate"]
                   )

# Information on the data frame
print("Number of customers: ", df["Customer ID"].nunique())
print("Name of columns: ", df.columns)
print("Number of features(colomns):", len(df.columns))
print("shape of the data frame: ", df.shape)


Number of customers:  4372
Name of columns:  Index(['Invoice', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'Price', 'Customer ID', 'Country'],
      dtype='object')
Number of features(colomns): 8
shape of the data frame:  (541910, 8)


## Cleaning the data
Now we will try to clean the data, deleting lines without customers or resigned transactions and to remove duplicate values. We will also add a column "Totalprice" as the multiplication of the quantity by the unity prices.

In [2]:
# delete line without Customer id
df = df.dropna(subset=["Customer ID"])
print("After drop nan Customer ID :", df.shape)

# delete not finish transaction and negative quantity
mask_return = df["Quantity"] < 0
mask_cancel = df["Invoice"].str.startswith("C")
df = df[~(mask_return | mask_cancel)]
print("After deleting unfinished transaction/neg. quantity :" , df.shape)

# delete duplicate line
df = df.drop_duplicates()
print("After drop_duplicates :", df.shape)

# columns "TotalPrice"
df["TotalPrice"]=df["Quantity"] * df["Price"]
print("After adding 'TotalPrice': ",df.shape)

After drop nan Customer ID : (406830, 8)
After deleting unfinished transaction/neg. quantity : (397925, 8)
After drop_duplicates : (392733, 8)
After adding 'TotalPrice':  (392733, 9)


In [5]:
print("Data frame: \n", df.head(20))

Data frame: 
    Invoice StockCode                          Description  Quantity  \
0   536365    85123A   WHITE HANGING HEART T-LIGHT HOLDER         6   
1   536365     71053                  WHITE METAL LANTERN         6   
2   536365    84406B       CREAM CUPID HEARTS COAT HANGER         8   
3   536365    84029G  KNITTED UNION FLAG HOT WATER BOTTLE         6   
4   536365    84029E       RED WOOLLY HOTTIE WHITE HEART.         6   
5   536365     22752         SET 7 BABUSHKA NESTING BOXES         2   
6   536365     21730    GLASS STAR FROSTED T-LIGHT HOLDER         6   
7   536366     22633               HAND WARMER UNION JACK         6   
8   536366     22632            HAND WARMER RED POLKA DOT         6   
9   536368     22960             JAM MAKING SET WITH JARS         6   
10  536368     22913          RED COAT RACK PARIS FASHION         3   
11  536368     22912       YELLOW COAT RACK PARIS FASHION         3   
12  536368     22914         BLUE COAT RACK PARIS FASHION      

#### Some goals for this project:
churn risk,RFM (Recency-Frequency-Monetary), tenure, panier moyen, determinate also the product that was the most sell in term of quantity and after in term of money bring to the campany. 

## Behavioral Variables (RFM)

This variables define the behavior of a purchase of a customer. There are:

- R(Recency): numbers of days since the last command. More this value is low, more the customer is recent and potentially engaged the customer is.
- F(Frequency): total number of purchases the customer did in the period. This help to measure the faithfulness of the customer.
- M(Monetay): total amount spent by the customer in this period. show the financial value of thar client for the company.

Let's start.

In [23]:
# Date from which we will compute the number of days since the last command
date_limit=max(df["InvoiceDate"]) + pd.Timedelta(days=1)
date_limit=pd.Timestamp(date_limit).day_of_year # the number of the day in the year

print("Date limit: ",date_limit)

#Building recency
recency= (
    df.groupby("Customer ID")["InvoiceDate"] # we group by customer and toke only their invoicedate
    .max() #we select their greatest invoicedate, that means their last incode dat
    .apply( lambda last_date: (date_limit-pd.Timestamp(last_date).day_of_year)) # we compute the difference
    .reset_index() # to mage it a dataframe with two columns instead of one
    .rename(columns={"InvoiceDate":"Recency"}) # change the name of the column into Recency
)

# Display
print("Recency data frame:\n",recency)



Date limit:  344
Recency data frame:
      Customer ID  Recency
0          12346      326
1          12347        3
2          12348       76
3          12349       19
4          12350      311
...          ...      ...
4334       18280      278
4335       18281      181
4336       18282        8
4337       18283        4
4338       18287       43

[4339 rows x 2 columns]


Let's say that if a customer did not return since 100 days, we could consider him like . Personaly I don't see what is the importance of ML method like K mean method here. 

In [ ]:
frequency=( 
    df.groupby("Customer ID")["Invoice"]
    .count() #we count the number of invoice each customer have
    .reset_index()
    .rename(columns={"Invoice":"Frequency"})
)

print("Frequency data frame:\n",frequency)

Frequency data frame:
      Customer ID  Frequency
0          12346          1
1          12347        182
2          12348         31
3          12349         73
4          12350         17
...          ...        ...
4334       18280         10
4335       18281          7
4336       18282         12
4337       18283        721
4338       18287         70

[4339 rows x 2 columns]


In [29]:
monetary=(
    df.groupby("Customer ID")["TotalPrice"]
    .sum() # we make the sum of each totalprice for each customer
    .reset_index()
    .rename(columns={"TotalPrice":"Monetary"})
)

print("Monetary data frame:\n",monetary)

Monetary data frame:
      Customer ID  Monetary
0          12346  77183.60
1          12347   4310.00
2          12348   1797.24
3          12349   1757.55
4          12350    334.40
...          ...       ...
4334       18280    180.60
4335       18281     80.82
4336       18282    178.05
4337       18283   2045.53
4338       18287   1837.28

[4339 rows x 2 columns]


In [31]:
#we're going to concatenate all this data frame into one n the RFM data frame.
rfm=recency.merge(frequency,on="Customer ID")\
        .merge(monetary, on="Customer ID")

print("RFM data frame:\n",rfm)

RFM data frame:
      Customer ID  Recency  Frequency  Monetary
0          12346      326          1  77183.60
1          12347        3        182   4310.00
2          12348       76         31   1797.24
3          12349       19         73   1757.55
4          12350      311         17    334.40
...          ...      ...        ...       ...
4334       18280      278         10    180.60
4335       18281      181          7     80.82
4336       18282        8         12    178.05
4337       18283        4        721   2045.53
4338       18287       43         70   1837.28

[4339 rows x 4 columns]


### Scoring
Now we will try to make the different groups using a scoring method.

Let's choose 5 groups and say that:
- for Recency: 1:the oldest customers(with older purchase dates) , 5: the regular people or the new ones
- for Frequency: 1: the customer with lower number of purchase, 5: our regular customers
- for Monetary: 1: customers who spent less money , 5: customer who spent much money

In [43]:
rfm["R_score"]=pd.qcut(rfm["Recency"],5,[5,4,3,2,1])
rfm["F_score"]=pd.qcut(rfm["Frequency"],5,[1,2,3,4,5])
rfm["M_score"]=pd.qcut(rfm["Monetary"],5,[1,2,3,4,5])

print(rfm)

     Customer ID  Recency  Frequency  Monetary R_score F_score M_score
0          12346      326          1  77183.60       1       1       5
1          12347        3        182   4310.00       5       5       5
2          12348       76         31   1797.24       2       3       4
3          12349       19         73   1757.55       4       4       4
4          12350      311         17    334.40       1       2       2
...          ...      ...        ...       ...     ...     ...     ...
4334       18280      278         10    180.60       1       1       1
4335       18281      181          7     80.82       1       1       1
4336       18282        8         12    178.05       5       1       1
4337       18283        4        721   2045.53       5       5       4
4338       18287       43         70   1837.28       3       4       4

[4339 rows x 7 columns]


Next, let's define 3 greats classes for our customers, the VIP, the regular ones and the ...


talk also about the different marketing technics that we could applu to thos different classes.

Now we will  build a machine learning model to find some correlation or some group amoung this customers.

In [33]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler


In [ ]:
#Normalization of the data: to give to each column the same weight during the learning step
scaler=StandardScaler()
rfm_scaled=scaler.fit_transform(rfm[["Recency","Frequency","Monetary"]]) # we scale only this three columns
rfm_scaled=pd.DataFrame(rfm_scaled, columns=["Recency_norm", "Frequency_norm","Monetary_norm"]) #we add column names
rfm_scaled=pd.concat([rfm["Customer ID"], rfm_scaled], axis=1) # we just add the customer IDs


# Building the model
km_model=KMeans(int=4).fit()

In [42]:
print(rfm_scaled)

     Customer ID  Recency_norm  Frequency_norm  Monetary_norm
0          12346      2.703345       -0.396969       8.363977
1          12347     -0.890450        0.405730       0.251779
2          12348     -0.078230       -0.263925      -0.027939
3          12349     -0.712429       -0.077664      -0.032357
4          12350      2.536451       -0.326012      -0.190780
...          ...           ...             ...            ...
4334       18280      2.169283       -0.357056      -0.207901
4335       18281      1.090031       -0.370360      -0.219009
4336       18282     -0.834819       -0.348186      -0.208185
4337       18283     -0.879324        2.796087      -0.000299
4338       18287     -0.445398       -0.090968      -0.023482

[4339 rows x 4 columns]
